In [ ]:
import pandas as pd
import os
import warnings
import requests
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import zipfile
from urllib.request import urlopen
import shutil
from bs4 import BeautifulSoup as soup 
from time import sleep

In [ ]:
# Creamos una carpeta para que contenga a nuestro dataset
if not os.path.isdir('cbs/'): os.mkdir('cbs/')
# Movemos el directorio activo a esa localización
os.chdir('cbs/')

# https://stackoverflow.com/questions/67284489/how-to-load-a-csv-file-from-zipped-folder-from-url-into-pandas-dataframe
# descarga de URL y extracción de zip de los datos del año 2016 al 2017
for i in range(6, 8):
    file_name = '201'+ str(i) +'-capitalbikeshare-tripdata.zip'
    url= 'https://s3.amazonaws.com/capitalbikeshare-data/201'+ str(i) +'-capitalbikeshare-tripdata.zip'
    with urlopen(url) as response, open(file_name, 'wb') as out_file:
        shutil.copyfileobj(response, out_file)
        # extracting required file from zipfile
        with zipfile.ZipFile(file_name) as zf:
            zf.extractall()
    # deleting the zipfile from the directory
    os.remove(file_name)

# descarga de URL y extracción de zip de los datos del año 2018 al 2021 (cambia el nombre del archivo, diferente a los años anteriores)
for year in range(2018, 2022):
    for i in range(1, 13):
        file_name = str(year)+'{0:02d}-capitalbikeshare-tripdata.zip'.format(i)
        url= 'https://s3.amazonaws.com/capitalbikeshare-data/'+str(year)+'{0:02d}-capitalbikeshare-tripdata.zip'.format(i)
        with urlopen(url) as response, open(file_name, 'wb') as out_file:
            shutil.copyfileobj(response, out_file)
            # extracting required file from zipfile
            with zipfile.ZipFile(file_name) as zf:
                zf.extractall()
        # deleting the zipfile from the directory
        os.remove(file_name)

# descarga de URL y extracción de zip de los datos del año 2022 hasta el mes de octubre incluido (el anterior bucle da error en el 2022 por la falta del mes 11 y 12)
for i in range(1, 11):
    file_name = str(year)+'2022{0:02d}-capitalbikeshare-tripdata.zip'.format(i)
    url= 'https://s3.amazonaws.com/capitalbikeshare-data/2022{0:02d}-capitalbikeshare-tripdata.zip'.format(i)
    with urlopen(url) as response, open(file_name, 'wb') as out_file:
        shutil.copyfileobj(response, out_file)
        # extracting required file from zipfile
        with zipfile.ZipFile(file_name) as zf:
            zf.extractall()
    # deleting the zipfile from the directory
    os.remove(file_name)

C:\Users\Carlos\AppData\Local\Temp\ipykernel_14200\2829372594.py:49: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  cbs = pd.concat([pd.read_csv(elem) for elem in os.listdir() if elem.endswith(".csv")]).reset_index(drop=True)


In [ ]:
# Movemos el directorio activo a esa localización
os.chdir('cbs/')

# Leemos todos los archivos en una línea
cbs = pd.concat([pd.read_csv(elem) for elem in os.listdir() if elem.endswith(".csv") and elem.startswith("2")]).reset_index(drop=True)

C:\Users\Carlos\AppData\Local\Temp\ipykernel_20056\1183006145.py:5: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  cbs = pd.concat([pd.read_csv(elem) for elem in os.listdir() if elem.endswith(".csv") and elem.startswith("2")]).reset_index(drop=True)


In [ ]:
def washington_dc_weather(date): # https://github.com/hristo-mavrodiev/Webscrape_weather_data/blob/master/Freemeteo_scraping_hourly.ipynb
    """Function to webscrape one day from freemeteo.bg at London station.
    Input: 'year-month-day' -- 2019-01-01
    Output: pandas dataframe
    
    """
    url = F"https://freemeteo.bg/weather/Washington-D.C./history/daily-history/?gid=9036777&station=19056&date={date}&language=english&country=us-united-states"
    res =  requests.get(url,headers={"User-Agent":"Chrome/39.0.2171.95"})
    soup1 = soup(res.text, "lxml")
    #soup1
    columns = ['datetime','temperature','wind_dir','humidity','pressure','icon','description']
    df=pd.DataFrame(columns=columns)
    for row in soup1.findAll("tr"):
        cells = row.findAll("td")
        row = []
        if len(cells) ==10:
            time = cells[0].find(text=True)
            temperature = cells[1].find(text=True)
            #t2 = cells[2].find(text=True)
            wind_dir = cells[3].text
            #wind_gust = cells[4].find(text=True)
            humidity = cells[5].find(text=True)
            #dew = cells[6].find(text=True)
            pressure = cells[7].find(text=True)
            icon = cells[8].span.attrs['data-icon']
            date = f'{date}T{time}'
            description = cells[9].find('span', "details").text
            #print(cells)
            #print (date,t1,t2,wind_dir,wind_gust,hum,dew,pressure,icon,other)
            row.append([date,temperature,wind_dir,humidity,pressure,icon,description])
            row = pd.DataFrame(row, columns=columns)
            df = df.append(row, ignore_index=True)
 
    df.datetime = pd.to_datetime(df.datetime)
    return df

In [ ]:
def multi_weather(start,end):
    """
    Function to generate dates from start end dates.
    After the data-range is generated a webscrape for weather data is performed.
    
    Input:
        start: a start date string like '2019-01-01'
        end:' a end date string like 2019-08-01'
    Output: pandas dataframe with weather data
    """
    range_df = pd.period_range(start, end, freq='D').strftime('%Y-%m-%d')
    columns = ['datetime','temperature','wind_dir','humidity','pressure','icon','description']
    df=pd.DataFrame(columns=columns)    
    for range_date in (range_df):
        print (range_date)
        #input_date = i
        chunk = washington_dc_weather(range_date)
        sleep(0.1)
        df = df.append(chunk, ignore_index=True,sort=True)
        df = df[['datetime','temperature','wind_dir','humidity','pressure','icon','description']]
    #df.to_csv(f'{TIMEFRAME}/sds_bme/{start}_{end}_sds+bme_sof_{TIMEFRAME}.csv')
    return df

In [ ]:
def cleaning(df):
    """Function to clean webscraped data"""
    df.temperature = df.temperature.str.replace('°C','')
    df.humidity = df.humidity.str.replace('%','')
    df.pressure = df.pressure.str.replace('mb','')
    df.wind_dir = df.wind_dir.str.replace('Variable at ','NaN°')
    df[["wind_dir_deg",'wind_speed']] = df['wind_dir'].str.split("°",expand = True)
    df.wind_speed = df['wind_speed'].str.split(" ",expand = True)[0]
    df = df.drop(['wind_dir'],axis=1)
    return df[['datetime','temperature','humidity','pressure','icon','wind_dir_deg','wind_speed','description']]

In [ ]:
weather_data = cleaning(multi_weather('2016-01-01', '2022-11-01'))
weather_data.to_csv('weatherdata_wdc.csv')

2016-01-01


c:\Users\Carlos\Documents\Formación\Upgrade Hub Bootcamp\samplerepo\analytics\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname T identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


2016-01-02


c:\Users\Carlos\Documents\Formación\Upgrade Hub Bootcamp\samplerepo\analytics\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname T identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


2016-01-03


...

c:\Users\Carlos\Documents\Formación\Upgrade Hub Bootcamp\samplerepo\analytics\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname T identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


2022-10-31


c:\Users\Carlos\Documents\Formación\Upgrade Hub Bootcamp\samplerepo\analytics\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname T identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


2022-11-01


c:\Users\Carlos\Documents\Formación\Upgrade Hub Bootcamp\samplerepo\analytics\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname T identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [ ]:
weather_data.head()

,datetime,temperature,humidity,pressure,icon,wind_dir_deg,wind_speed,description
0,2016-01-01 00:58:00,7,61,1020.0,4,NaN,2,Cloudy at 1400m 6-Hour Maximum Temperature::...
1,2016-01-01 01:58:00,6,66,1020.0,3,320,7,Broken Clouds at 1500m
2,2016-01-01 02:58:00,6,61,1020.0,1,320,6,Clear below 3700m
3,2016-01-01 03:58:00,5,66,1020.0,1,320,7,Clear below 3700m 3-Hour Pressure Decreasing...
4,2016-01-01 04:58:00,4,70,1020.0,2,Calm,None,Few Clouds at 5200m


In [ ]:
# redondeo las filas del tiempo según los minutos, a la hora
weather_data["hour_datetime"] = weather_data["datetime"].dt.round("H") # https://stephenallwright.com/pandas-datetime-round-to-hour/
# elimino los valores duplicados según la columna hora
weather_data.drop_duplicates(subset=['hour_datetime'], inplace= True)